### Chapter 13 - Custom User Model:

The biggest problem with the auth.user is that it is very hard to modify once you migrate, also in django documentation it is said that we should avoid any changes.<br>

The solution is to use a CustomUser model.<br>
Please understand that in order to use a CustomUser model you should be in the early stage of you project, meaning that you shouldnt have done any migrations otherwise it is almost imposible to achieve.<br>

So, lets start with a new project:

### BookStore:
1. Create a new django project called 'BookStore'
2. Add a new app called 'accounts' to the project
3. Add the new add to the INSTALLED_APP list (settings.py)
4. Navigate to account.models.py
5. From django.contrib.auth.models import AbstractUser
6. Create a new model called CustomUser and extend it from AbstractUser class

In [ ]:
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    pass

Now we need to tell django that it should use this model instead of its existing auth model

7. Navigate to settings.py and add a new variable called AUTH_USER_MODEL and pass 'accounts.CustomUser' to it

In [ ]:
AUTH_USER_MODEL = 'accounts.CustomUser'

Lets talk about AbstractUser Class:

The abstract user class is a class that already has some fields in it, you can use it and also add more fields to it.

Here are the fields included in the AbstractUser class:

- username: A string field for the username, which is unique.
- first_name: A string field for the user's first name.
- last_name: A string field for the user's last name.
- email: An email field for the user's email address.
- password: A string field for storing the hashed password.
- groups: A many-to-many relationship to the Group model, representing the groups the user belongs to.
- user_permissions: A many-to-many relationship to the Permission model, representing the permissions granted to the user.
- is_staff: A boolean field indicating whether the user can access the admin site.
- is_active: A boolean field indicating whether the user account is active.
- is_superuser: A boolean field indicating whether the user has all permissions without explicitly assigning them.
- last_login: A datetime field for the last time the user logged in.
- date_joined: A datetime field for when the user account was created.

Now lets add a new field to it just for seeing how it is:

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    age = models.PositiveIntegerField()

PositiveIntegerField only accepts Possitive values

Before we mode on, lets talk about the difference between AbstractUser and AbstractBaseUser classes:


### BastractUser: 

In Django, both AbstractUser and AbstractBaseUser are classes used to create custom user models, but they serve different purposes and offer different levels of customization. Here’s a detailed comparison of the two:
AbstractUser

- Purpose:

    - AbstractUser is a more complete and feature-rich base class for creating custom user models.
    - It provides all the fields and methods commonly needed for a user model out of the box.

- Fields:

    - Includes common fields such as username, first_name, last_name, email, password, groups, user_permissions, is_staff, is_active, is_superuser, last_login, and date_joined.

- Use Case:

    - Use AbstractUser if you need to extend the default Django user model with additional fields or methods, but want to retain all the built-in functionality.

### AbstractBaseUser

- Purpose:

    - AbstractBaseUser provides the bare minimum for a custom user model, offering a lot more flexibility for defining a completely custom user model.
    - It includes only the core authentication functionality, without any of the fields or methods provided by AbstractUser.

- Fields:

    - Includes only the password, last_login, and is_active fields.
    - Requires you to define additional fields such as username or email, and manage user authentication and permissions more manually.

- Use Case:

    - Use AbstractBaseUser if you need to build a custom user model from scratch, with completely different fields and authentication logic than what is provided by Django’s default user model.

### Now lets add some constraints to our model:

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    age = models.PositiveIntegerField(null=True, blank=True) 

In Django, null and blank are two model field options that control different aspects of data validation and database schema.<br>

Null is for database and none is for the form that user is going to fillin.

Please understand that for a charfield a "" is considered blank but not null.